# Model Inference

## 1. Installs and Imports

In [ ]:
!pip install datasets transformers seqeval

     |████████████████████████████████| 306 kB 3.0 MB/s 
     |████████████████████████████████| 3.4 MB 12.4 MB/s 
     |████████████████████████████████| 43 kB 1.2 MB/s 
     |████████████████████████████████| 243 kB 62.5 MB/s 
     |████████████████████████████████| 132 kB 32.5 MB/s 
     |████████████████████████████████| 61 kB 344 kB/s 
     |████████████████████████████████| 1.1 MB 53.5 MB/s 
     |████████████████████████████████| 3.3 MB 57.7 MB/s 
     |████████████████████████████████| 895 kB 40.5 MB/s 
     |████████████████████████████████| 596 kB 44.5 MB/s 
     |████████████████████████████████| 160 kB 57.8 MB/s 
     |████████████████████████████████| 192 kB 53.2 MB/s 
     |████████████████████████████████| 271 kB 68.9 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16181 sha256=2afa5e2eaa393d2652c672640cdb9a114d1a2b622a6711bb12660e9ff8714e38
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72

In [1]:
import os
import random
import transformers
import pandas as pd
import numpy as np
from datetime import date
from pathlib import Path
from google.colab import drive
from seqeval.metrics import accuracy_score
from IPython.display import display, HTML
from collections import defaultdict, Counter, OrderedDict
from datasets import load_dataset, load_metric, load_from_disk, ClassLabel, Sequence
from transformers import TrainingArguments, Trainer, AutoTokenizer, AutoModelForTokenClassification, DataCollatorForTokenClassification

#scraping
import requests
from bs4 import BeautifulSoup

import spacy
from spacy import displacy

#inference
import torch
from transformers import pipeline

print(transformers.__version__)

In [ ]:
system = "COLAB" #["AWS", "COLAB"]

In [ ]:
if system=="AWS":
    fs = s3fs.S3FileSystem()    
    s3_bucket = "govuk-data-infrastructure-integration"
    DATA_DIR = f's3://{s3_bucket}/model-data/govner-data'
    for f in fs.ls(DATA_DIR):
        print(f)
    #Manage interactions with the Amazon SageMaker APIs and any other AWS services needed.
    # sagemaker session bucket -> used for uploading data, models and logs
    # sagemaker will automatically create this bucket if it not exists
    sess = sagemaker.Session() 
    sagemaker_session_bucket= s3_bucket
    if sagemaker_session_bucket is None and sess is not None:
        # set to default bucket if a bucket name is not given
        sagemaker_session_bucket = sess.default_bucket()
        
    role = sagemaker.get_execution_role()
    sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

    print(f"sagemaker role arn: {role}")
    print(f"sagemaker bucket: {sess.default_bucket()}")
    print(f"sagemaker session region: {sess.boto_region_name}")
elif system=="COLAB":
    drive.mount("/content/gdrive")
    DATA_DIR = os.path.join("/content/gdrive/Shared drives/", "GOV.UK teams/2020-2021/Data labs/content-metadata-2021/Data")
    MODEL_DIR = os.path.join("/content/gdrive/Shared drives/", "GOV.UK teams/2020-2021/Data labs/content-metadata-2021/Models")
    RESULTS_DIR = os.path.join("/content/gdrive/Shared drives/", "GOV.UK teams/2020-2021/Data labs/content-metadata-2021/Models/Metrics")

Mounted at /content/gdrive


In [ ]:
print("Data Folder: {}".format(DATA_DIR))
print(os.listdir(DATA_DIR)[:3])
print("Model Folder: {}".format(MODEL_DIR))
print(os.listdir(MODEL_DIR)[:3])

Data Folder: /content/gdrive/Shared drives/GOV.UK teams/2020-2021/Data labs/content-metadata-2021/Data
['label_map_09062020_more_ents.json', 'label_map_12062020_more_ents.json', 'line_by_line_NER_data_sampled_09062020_more_ents.csv']
Model Folder: /content/gdrive/Shared drives/GOV.UK teams/2020-2021/Data labs/content-metadata-2021/Models
['distilbert-base-uncased-finetuned-ner-conll2003', 'distilbert-base-uncased-finetuned-ner-govuk', 'distilbert-base-uncased-finetuned-ner-govuk-SAMPLED-08-12-2021']


## 2. Scrape Govuk Content

In [ ]:
def get_page_soup(url):
  page = requests.get(url)
  soup = BeautifulSoup(page.content, "html.parser")
  return soup

In [ ]:
def get_sents_from_soup(soup):
  body = soup.findAll(attrs={"class":"gem-c-govspeak"})
  sents = [i.text.split('\n') for i in body]
  sents_clean = [list(filter(None, i)) for i in sents]
  return sents_clean

In [ ]:
def url_get_sents(url):
  soup = get_page_soup(url)
  sents_clean = get_sents_from_soup(soup)
  return sents_clean

In [ ]:
result = url_get_sents('https://www.gov.uk/student-visa')

In [ ]:
result

[['You can apply for a Student visa to study in the UK if you’re 16 or over and you:',
  'have been offered a place on a course by a licensed student sponsor',
  'have enough money to support yourself and pay for your course - the amount will vary depending on your circumstances',
  'can speak, read, write and understand English',
  'have consent from your parents if you’re 16 or 17 - you’ll need evidence of this when you apply',
  'If you’re 16 or 17 and you want to study at an independent school in the UK, you may be eligible for a Child Student visa instead.',
  'This visa has replaced the Tier 4 (General) student visa.',
  'If you or your family are from the EU, Switzerland, Norway, Iceland or Liechtenstein',
  'If you or your family member started living in the UK by 31 December 2020, you may be able to apply to the free EU Settlement Scheme.',
  'The deadline to apply was 30 June 2021 for most people. You can still apply if either:',
  'you have a later deadline - for example, yo

## 3. Do Inference

### 3A. Load Model

Load model from local


In [ ]:
MODEL_DIR 
model_name = "distilbert-base-uncased"
task = "ner"
dataset_name = "govuk"
req_date = "13-12-2021"
dataset_type = 'FULL'
chkpoint = 'checkpoint-73500'

In [ ]:
OUTPUT_PATH = f"{MODEL_DIR}/{model_name}-finetuned-{task}-{dataset_name}-{dataset_type}-{req_date}/{chkpoint}"
OUTPUT_PATH

'/content/gdrive/Shared drives/GOV.UK teams/2020-2021/Data labs/content-metadata-2021/Models/distilbert-base-uncased-finetuned-ner-govuk-FULL-13-12-2021/checkpoint-73500'

In [ ]:
os.listdir(OUTPUT_PATH)

['config.json',
 'pytorch_model.bin',
 'tokenizer_config.json',
 'special_tokens_map.json',
 'vocab.txt',
 'tokenizer.json',
 'training_args.bin',
 'optimizer.pt',
 'scheduler.pt',
 'trainer_state.json',
 'rng_state.pth']

Load model and tokeniser

In [ ]:
from transformers import AutoModelForTokenClassification, AutoTokenizer
import torch

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(OUTPUT_PATH)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(OUTPUT_PATH)

### 3B. Hugging Face Pipelines

Use Huggingface Pipelines

In [ ]:
sequence = "You must be at least 17 years old to have a drivers licence " \
"failure to provide this certificate will mean imprisonment in the UK and barring from countries like EU and US"

In [ ]:
# set up pipeline with model and tokeniser
token_classifier = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

In [ ]:
print(sequence)
print(len(sequence))

You must be at least 17 years old to have a drivers licence failure to provide this certificate will mean imprisonment in the UK and barring from countries like EU and US
170


In [ ]:
result = token_classifier(sequence)

In [ ]:
result

[{'end': 33,
  'entity_group': 'STATE',
  'score': 0.6916776,
  'start': 21,
  'word': '17 years old'},
 {'end': 51,
  'entity_group': 'PER',
  'score': 0.99765354,
  'start': 44,
  'word': 'drivers'},
 {'end': 59,
  'entity_group': 'FORM',
  'score': 0.9795044,
  'start': 52,
  'word': 'licence'},
 {'end': 95,
  'entity_group': 'FORM',
  'score': 0.98986244,
  'start': 84,
  'word': 'certificate'},
 {'end': 128,
  'entity_group': 'LOC',
  'score': 0.9997448,
  'start': 122,
  'word': 'the uk'},
 {'end': 155,
  'entity_group': 'LOC',
  'score': 0.99967575,
  'start': 146,
  'word': 'countries'},
 {'end': 163,
  'entity_group': 'LOC',
  'score': 0.9998914,
  'start': 161,
  'word': 'eu'},
 {'end': 170,
  'entity_group': 'LOC',
  'score': 0.99533564,
  'start': 168,
  'word': 'us'}]

Now try with gov.uk outputs

In [ ]:
page_sents = url_get_sents(url='https://www.gov.uk/marriage-visa/documents-you-will-need')

In [ ]:
page_sents

[['You must provide a passport or travel document. Your passport should be valid for the whole of your stay in the UK and contain a blank page for your visa.',
  'You can supply the following to support your application:',
  'details of the marriage or civil partnership and proof that you’ve paid money for some of its costs',
  'proof that you’re planning to get married in the UK, for example a booking confirmation or emails between you and the venue',
  'See the full list of documents you can provide to prove your eligibility.',
  'You’ll need to provide a certified translation of any documents that are not in English or Welsh.',
  'If you’ve been married before',
  'You’ll need to show proof that you’re free to marry or enter into a civil partnership again, for example a:',
  'decree absolute',
  'death certificate of a previous partner',
  'You may need to provide additional documents depending on your circumstances.']]

In [ ]:
sent_len = 0
ners = []
for i in page_sents[0]:
  result = token_classifier(i)
  for j in result:
    j['start'] += sent_len
    j['end'] += sent_len
  sent_len += len(i) + 1
  ners.append(result)

In [ ]:
ners_flat = [item for sublist in ners for item in sublist]

In [ ]:
ners_flat

[{'end': 27,
  'entity_group': 'FORM',
  'score': 0.99917954,
  'start': 19,
  'word': 'passport'},
 {'end': 37,
  'entity_group': 'EVENT',
  'score': 0.99745935,
  'start': 31,
  'word': 'travel'},
 {'end': 61,
  'entity_group': 'FORM',
  'score': 0.9692217,
  'start': 38,
  'word': 'document. your passport'},
 {'end': 77,
  'entity_group': 'STATE',
  'score': 0.99483514,
  'start': 72,
  'word': 'valid'},
 {'end': 114,
  'entity_group': 'LOC',
  'score': 0.9943054,
  'start': 108,
  'word': 'the uk'},
 {'end': 153,
  'entity_group': 'FORM',
  'score': 0.98904836,
  'start': 144,
  'word': 'your visa'},
 {'end': 211,
  'entity_group': 'FORM',
  'score': 0.9993825,
  'start': 200,
  'word': 'application'},
 {'end': 236,
  'entity_group': 'STATE',
  'score': 0.99277097,
  'start': 228,
  'word': 'marriage'},
 {'end': 257,
  'entity_group': 'STATE',
  'score': 0.99308074,
  'start': 240,
  'word': 'civil partnership'},
 {'end': 267,
  'entity_group': 'FORM',
  'score': 0.927255,
  'start

In [ ]:
res_ents = [(i['entity_group'], i['start'], i['end']) for i in ners_flat]
res_ents

[('FORM', 19, 27),
 ('EVENT', 31, 37),
 ('FORM', 38, 61),
 ('STATE', 72, 77),
 ('LOC', 108, 114),
 ('FORM', 144, 153),
 ('FORM', 200, 211),
 ('STATE', 228, 236),
 ('STATE', 240, 257),
 ('FORM', 262, 267),
 ('FINANCE', 280, 290),
 ('FINANCE', 307, 312),
 ('STATE', 347, 354),
 ('LOC', 358, 364),
 ('CONTACT', 388, 400),
 ('CONTACT', 404, 410),
 ('LOC', 431, 436),
 ('FORM', 458, 467),
 ('FORM', 536, 557),
 ('FORM', 565, 574),
 ('LOC', 602, 607),
 ('STATE', 624, 631),
 ('DATE', 632, 638),
 ('FORM', 659, 664),
 ('STATE', 707, 724),
 ('EVENT', 754, 762),
 ('FORM', 769, 780),
 ('PER', 795, 802),
 ('FORM', 838, 847),
 ('STATE', 866, 879)]

Stitch sents into one 'doc'

In [ ]:
seq = ' '.join(page_sents[0])

In [ ]:
seq

'You must provide a passport or travel document. Your passport should be valid for the whole of your stay in the UK and contain a blank page for your visa. You can supply the following to support your application: details of the marriage or civil partnership and proof that you’ve paid money for some of its costs proof that you’re planning to get married in the UK, for example a booking confirmation or emails between you and the venue See the full list of documents you can provide to prove your eligibility. You’ll need to provide a certified translation of any documents that are not in English or Welsh. If you’ve been married before You’ll need to show proof that you’re free to marry or enter into a civil partnership again, for example a: decree absolute death certificate of a previous partner You may need to provide additional documents depending on your circumstances.'

In [ ]:
colors = {"ORG": "#7c5cdd", 
          "FORM": "#26e21c",
          "LOC": "#eee65c",
          "MONEY": "#80bab2",
          "SCHEME": "#b76d14",
          "DATE": "#bc8251",
          "STATE": "#bd4c33",
          "PER": "#c0970b",
          "FINANCE": "#debdd8",
          "FORM": "#48aba2",
          "EVENT": "#0a8dd9",
          "CONTACT": "#807388"}

In [ ]:
def display_entities(text, entities):
  nlp = spacy.blank("en")
  doc = nlp(text)
  ents = []
  for ee in entities:
      ents.append(doc.char_span(ee[1], ee[2], ee[0]))
  doc.ents = ents
  options={'distance': 90, 'colors':colors}
  displacy.render(doc, style='ent', jupyter=True, options=options)

In [ ]:
display_entities(text=seq, entities=res_ents)

In [ ]:
def get_ners_and_flatten(sents):
  sent_len = 0
  ners = []
  for i in sents[0]:
    result = token_classifier(i)
    for j in result:
      j['start'] += sent_len
      j['end'] += sent_len
    sent_len += len(i) + 1
    ners.append(result)
  ners_flat = [item for sublist in ners for item in sublist]
  res_ents = [(i['entity_group'], i['start'], i['end']) for i in ners_flat]
  return res_ents

In [ ]:
def stitch_sents(sent_list):
  seq = ' '.join(sent_list)
  return seq

In [ ]:
def display_entities(text, entities):
  nlp = spacy.blank("en")
  doc = nlp(text)
  ents = []
  for ee in entities:
      ents.append(doc.char_span(ee[1], ee[2], ee[0]))
  doc.ents = ents
  options={'distance': 90, 'colors':colors}
  return displacy.render(doc, style='ent', jupyter=True, options=options)

## Pipeline

In [ ]:
def url_to_spacy_viz(url):
  print(url)
  sents = url_get_sents(url)
  flat_ners = get_ners_and_flatten(sents)
  stitched = stitch_sents(sents[0])
  disp_ents = display_entities(stitched, flat_ners)
  return disp_ents

In [ ]:
url_to_spacy_viz(url='https://www.gov.uk/hmrc-internal-manuals/tobacco-products-duty/tpd3180')

https://www.gov.uk/hmrc-internal-manuals/tobacco-products-duty/tpd3180


In [ ]:
paths = ['/student-visa', 
         '/marriage-visa', 
         '/marriage-visa/eligibility', 
         '/marriage-visa/documents-you-will-need', 
         '/marriage-visa/apply', 
         '/guidance/covid-19-coronavirus-restrictions-what-you-can-and-cannot-do#what-has-changed',
         ]

In [ ]:
for p in paths:
  url_p = f"http://www.gov.uk{p}"
  p_dash = p.replace("/", "_")
  fname = f"{DATA_DIR}/Images/img_{p_dash}"
  print(fname)
  disp_ents = url_to_spacy_viz(url_p)
  # output_path = Path(f"{DATA_DIR}/Images/img_{p_dash}.svg")
  # output_path.open("w", encoding="utf-8").write(disp_ents)

/content/gdrive/Shared drives/GOV.UK teams/2020-2021/Data labs/content-metadata-2021/Data/Images/img__student-visa
http://www.gov.uk/student-visa


/content/gdrive/Shared drives/GOV.UK teams/2020-2021/Data labs/content-metadata-2021/Data/Images/img__marriage-visa
http://www.gov.uk/marriage-visa


/content/gdrive/Shared drives/GOV.UK teams/2020-2021/Data labs/content-metadata-2021/Data/Images/img__marriage-visa_eligibility
http://www.gov.uk/marriage-visa/eligibility


/content/gdrive/Shared drives/GOV.UK teams/2020-2021/Data labs/content-metadata-2021/Data/Images/img__marriage-visa_documents-you-will-need
http://www.gov.uk/marriage-visa/documents-you-will-need


/content/gdrive/Shared drives/GOV.UK teams/2020-2021/Data labs/content-metadata-2021/Data/Images/img__marriage-visa_apply
http://www.gov.uk/marriage-visa/apply


/content/gdrive/Shared drives/GOV.UK teams/2020-2021/Data labs/content-metadata-2021/Data/Images/img__guidance_covid-19-coronavirus-restrictions-what-you-can-and-cannot-do#what-has-changed
http://www.gov.uk/guidance/covid-19-coronavirus-restrictions-what-you-can-and-cannot-do#what-has-changed
